<center>
<a href="https://websites.isae-supaero.fr/certificat/" ><img src="https://websites.isae-supaero.fr/IMG/gif/computer-science.gif" style="float:left; max-width: 120px; display: inline" alt="Toulouse Tech"/> </a>

<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:right; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

</center>

# [Certificat Science des Données](https://github.com/Certificat-sciences-des-donnees-bigdata) [Module de Sensibilisation](https://github.com/Certificat-sciences-des-donnees-bigdata/Module-sensibilisation)

# [Exploration Multidimensionnelle]()
# [Anayse en Composantes Principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf) et [Analyse Factorielle Discriminante](http://wikistat.fr/pdf/st-m-explo-afd.pdf) 
Ce notebook introduit l'exploration statistique en utilisant la librairie `scikit-learn` avec des exemples de mise en oeuvre de l'[ACP](http://wikistat.fr/pdf/st-m-explo-acp.pdf) et de l'[AFD](http://wikistat.fr/pdf/st-m-explo-afd.pdf). 

## 1 Introduction

L'objectif de ce tutoriel est d'apprendre l'exploration de données multidimensionnelles. Nous utiliserons la bibliothèque `Scikit-Learn` pour réaliser ces études. `Scikit-Learn` est une bibliothèque standard et populaire pour l'analyse de données. Elle est également largement utilisée pour l'analyse prédictive (apprentissage automatique). `Scikit-Learn` est une bibliothèque open-source. C'est la communauté Python qui maintient et développe cette boite à outils. Cette librairie est compatible avec les dataframe de `pandas` et avec les matrices de `numpy`.

Vous pouvez trouver toutes les informations utiles sur la [documentation](http://scikit-learn.org/stable/user_guide.html) de `Scikit-Learn`. La documentation comporte de nombreux exemples avec des jeux de données disponibles en ligne. Lire les exemples est un bon point de départ pour progresser (analyse de données, traitement du langage naturel, traitement d'images, données tabulaires etc...).

Aujourd'hui, nous allons nous concentrer sur l'analyse en composantes principales. Le module [ACP](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) centre les données par défaut mais il sera nécessaire de réduire les données de l'ACP. 

# <FONT COLOR="Red">Première partie: Exploration</font>

In [ ]:
# Importation des librairies que nous utiliserons.

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import discriminant_analysis

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patheffects as PathEffects

## 2.  ACP des données "jouet"

Nous commençons par créer un jeu de données simple pour comprendre les entrées et sorties du module `decomposition.ACP` de `Scikit-Learn`.

Ce jeu de données rassemble les notes en maths, français, physique et anglais de 9 lycéens.

Vous pouvez trouver cet exemple :

- [Introduction](http://wikistat.fr/pdf/st-l-des-multi) ACP par Wikistat.

- [Tutoriel](https://github.com/wikistat/Exploration/blob/master/TutosRudim/Cal1-R-SVDtoACP.ipynb) ACP avec R.


In [ ]:
notes = [
    [6, 6, 5, 5.5],
    [8, 8, 8, 8],
    [6, 7, 11, 9.5],
    [14.5, 14.5, 15.5, 15],
    [14, 14, 12, 12.5],
    [11, 10, 5.5, 7],
    [5.5, 7, 14, 11.5],
    [13, 12.5, 8.5, 9.5],
    [9, 9.5, 12.5, 12]
]

data = pd.DataFrame(
   notes,
   index = ["jean", "alai", "anni", "moni", "didi", "andr", "pier", "brig", "evel"],
   columns=["Math", "Phys", "Fran", "Angl"]
)

In [ ]:
data

In [ ]:
data.boxplot()
plt.show()

In [ ]:
# Matrice des nuages de points:

pd.plotting.scatter_matrix(
    data, # Input data.
    diagonal='kde', # Plot density functions
    figsize = (5, 5), # Size of the figure.
)

plt.show()

**Question:** Quel est la nature des liaisons entre les variables?

In [ ]:
# Matrice de variance covariance:
data.cov()

In [ ]:
# Matrice des corrélations:
data.corr()

**Question** Que pouvons-nous conclure de l'analyse de corrélation?

### 2.1 Valeurs propres de l'ACP non réduite

In [ ]:
# Calcul des valeurs propres ou variances des composantes principales.
pca = decomposition.PCA()

# L'ACP de Scikit-Learn n'est pas réduire par défaut. 
# En outre, le diviseur de la formule de calcul de la variance est celui d'une estimation sans 
# biais, i.e (n-1) au lieu de n.
valeurs_propres = pca.fit(data).explained_variance_

valeurs_propres

**Question**: Retrouver les valeurs propres du cours.

In [ ]:
valeurs_propres * 8/9

### 2.2 Vecteurs propres de l'ACP non réduite

In [ ]:
# Les vecteurs propres sont aussi les coefficients des combinaisons linéaires des variables 
# permettant de définir les variables principales.
pca.components_.T

### 2.3 Composantes principales de l'ACP non réduite

In [ ]:
components = pca.fit(data).transform(data)

print(components)

Distribution des composantes principales. 

**Question** Combien de composantes principales devons-nous choisir pour représenter les données?

Idéalement, nous aimerions trouver un bon compromis entre le nombre de dimensions et le niveau 
d'information restitué.

In [ ]:
plt.boxplot(components)

plt.show()

### 2.4 Représentation des individus

In [ ]:
plt.figure(figsize = (8,5))

for i, j, nom in zip(components[:,0], components[:,1], data.index):
    
    plt.text(i, j, nom, fontsize=16)
    
plt.axis((-10, 12, -6, 8))  
plt.show()

### 2.5 Graphe des variables
Calcul des coordonnées et réprésentation des variables

In [ ]:
# Attention

# Le signe d'un vecteur propre n'est pas déterminé parce qu'il s'agit d'une direction ou d'un 
# sous-espace qui est "propre" pour une matrice. Selon l'algorithme ou le logiciel utilisé, 
# le vecteur peut être orienté dans une direction ou dans l'autre mais c'est la même direction qui 
# est définie. Cela n'a aucune influence sur l'interprétation de ces résultats.

c1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
c2 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1)

for i, j, nom in zip(c1, c2, data.columns):
    plt.text(i, j, nom, fontsize=16)
    plt.arrow(0, 0, i, j, color='black')

plt.axis((-4,4,-4,4))
plt.show()

Le cercle des corrélations n'est pas tracé car les variables ne sont pas réduites.

## 3 AFD de données "jouet"
L'objectif de cet exemple sur des données très simples est de montrer l'intérêt d'une AFD par rapport à une ACP lorsque des groupes ou classes des individus sont connues *a priori*.

### 3.1 Lecture des données
Les données sont issues de l'observation de 6 variables, des dimensions d'ailes, élitres, antennes, pattes de 3 classes d'insectes. Il s'agit donc de vérifier graphiquement la bonne capacité de ces variables à distinguer ou discriminer ces trois classes.

In [ ]:
# Chargement des données:
insect = pd.read_csv("https://www.math.univ-toulouse.fr/~besse/Wikistat/Data/lubisch.txt", sep='\s+', header=0)
insect.head()

### 3.2 ACP 
**Question** Quel est le graphique ci-dessous? Comment choisir le nombre de composantes?

In [ ]:
Y = insect.pop("Y")

X = preprocessing.scale(insect)

C = pca.fit(X).transform(X)

plt.boxplot(C)

plt.show()

#### Graphique des individus

In [ ]:
plt.figure(figsize=(10,6))

for i, j, nom in zip(C[:,0],C[:,1], Y):

    plt.scatter(i, j, color=nom)
    
plt.axis((-5,7,-4,4))  
plt.show()

**Q** Commenter la forme des trois nuages de points, la déjà bonne séparation des classes.

#### Graphique des variables

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize=(5,5))

ax = fig.add_subplot(1, 1, 1)

for i, j, nom in zip(coord1, coord2, insect.columns):
    plt.text(i, j, nom, fontsize=16)
    plt.arrow(0, 0, i, j, color='black')

plt.axis((-1.2, 1.2, -1.2, 1.2))

cercle = plt.Circle((0,0), radius=1, color='gray', fill=False) # Cercle

ax.add_patch(cercle)

plt.show()

**Q** Commenter la structure de corrélation des variables, la qualité de représentation.

### 3.3 AFD


L'AFD est l'ACP des barycentres des classes avec la métrique de Mahalanobis dans l'espace des individus.

**Q** Comment cette métrique est-elle définie?

In [ ]:
method = discriminant_analysis.LinearDiscriminantAnalysis()
lda = method.fit(X,Y)

**Q** Que devient la forme des nuages? A quoi cela est-il dû? Commenter la séparation des classes.

In [ ]:
clda = lda.transform(X)

for i, j, nom in zip(clda[:,0], clda[:,1], Y):

    plt.scatter(i, j, color=nom)

plt.axis((-8,7,-4,6))  
plt.show()

# <FONT COLOR="Red">Compléments: Exploration</font> de données plus complexes
Les deux exemples qui suivents proposent des applications de l'ACP à d'autres jeux de données afin de montrer les capacités de cette méthode d'exploration multidimensionnelle. Ce sont des compléments intéressants mais pas indispensables à la bonne compréhension de la suite du cours.

## 4 ACP des données "Caractères"


Il s'agit d'explorer les données issues de la pixellisation de tracés de caractères dont les procédés d'obtention et prétraitement sont décrits sur le [site de l'UCI](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) (Lichman, 2013). Les chiffres ont été saisies sur des tablettes à l'intérieur de cadres de résolution $500\times 500$. Des procédures de normalisation,  ré-échantillonnage spatial puis de lissage ont été appliquées. Chaque caractère apparaît finalement discrétisé sous la forme d'une matrice $8\times 8$ de pixels à 16 niveaux de gris et identifié par un label. Les données sont archivées sous la forme d'une matrice ou tableau à trois indices. Elles sont également archivées après vectorisation des images sous la forme d'une matrice à $p=64$ colonnes.

L'étude du même type de données, mais nettement plus complexes (MNIST): 60 000 caractères représentés par des images de 784 pixels (26 $\times$ 26) fait l'objet d'un autre calepin.

### 4.1 Prise en main des données

In [ ]:
digits = datasets.load_digits()
print(digits)

In [ ]:
# Dimensions
digits.images.shape

In [ ]:
# Sous forme d'un cube d'images 1797 x 8x8
print(digits.images)

In [ ]:
# Sous forme d'une matrice 1797 x 64
print(digits.data)

In [ ]:
# Label réel de chaque caractère
print(digits.target)

Voici un aperçu des empilements des images à décrire puis ensuite en principe à discriminer:

In [ ]:
images_and_labels = list(zip(digits.images, digits.target))

for index, (image, label) in  enumerate(images_and_labels[:8]):
     plt.subplot(2, 4, index + 1)
     plt.axis('off')
     plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
     plt.title('Chiffres: %i' % label)

### 4.2 Analyse en composantes principales

In [ ]:
X = digits.data

y = digits.target

target_name = [0,1,2,3,4,5,6,7,8,9]

# définition de la commande
pca = decomposition.PCA()

# Estimation, calcul des composantes principales
C = pca.fit(X).transform(X)

# Décroissance de la variance expliquée
plt.plot(pca.explained_variance_ratio_)
plt.show()

Diagramme boîte des premières composantes principales.

In [ ]:
plt.boxplot(C[:,0:20])
plt.show()

**Q** Quelle dimension retenir en principe?

Représentation des caractères dans le premier plan principal. 

La représentation des variables (pixels) et le *biplot* n'ont pas grand intérêt pour ces données. 

In [ ]:
plt.scatter(C[:,0], C[:,1], c=y, label=target_name)
plt.show()

Le même graphique avec une légende mais moins de couleurs.

In [ ]:
# attention aux indentations
plt.figure()

for i, (c, label) in enumerate(zip("rgbcmykrgb", target_name)):
       plt.scatter(C[y == i, 0], C[y == i, 1], c=c, label=label)

plt.legend()
plt.title("ACP Digits")
plt.show()

Graphique en trois dimensions.

In [ ]:

fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], c=y, cmap=plt.cm.Paired)
ax.set_title("ACP: trois premieres composantes")
ax.set_xlabel("Comp1")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("Comp2")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("Comp3")
ax.w_zaxis.set_ticklabels([])
plt.show()

## 5  Données "cubiques" de l'OCDE
###  5.1 Introduction
#### Objectif
L'objectif de cette section  est l'exploration de données socio-économiques plus complexes. La principale spécificité de ces données est de se présenter  sous la forme d'un cube de données ou tableau à trois entrées: le numéro de ligne, le numéro de variable et l'année d'observation de cette variable. Après une description classique, la mise en oeuvre de l'analyse en composantes principales avec python nécessite un effort particulier afin de produire les graphes adaptés à la structure particulière des données. 

#### Les données
Les données sont issues de l'Observatoire de l'OCDE.  Pour chaque pays membre et pour chacune des années  1975, 1977, 1979, 1981, on connaît les valeurs prises par les  variables suivantes qui sont toutes des \emph{taux}~:
- Taux brut de natalité, 
- Taux de chômage, 
- Pourcentage d'actifs dans le secteur primaire, 
- Pourcentage d'actifs dans le secteur secondaire, 
- produit intérieur brut (par habitant), 
- Formation brute de capital fixe (par habitant), 
- Hausse des prix, 
- Recettes courantes  (par habitant), 
- Mortalité infantile, 
- Consommation de protéines animales  (par habitant), 
- Consommation d'énergie  (par habitant).

Elles sont disponibles dans le fichier: `ocdeR.dat`.

Les mêmes variables sont donc observées, sur les mêmes pays ou individus à quatre dates différentes. Plusieurs stratégies d'analyse sont possibles (tableau moyen, tableaux concaténés, meilleur compromis ou double ACP). La plus adaptée pour ces données est de considérer les observations des variables pour chacun des individus:  pays $\times$ années. 



### 5. 2 Lecture des données

In [ ]:
ocde = pd.read_csv("https://www.math.univ-toulouse.fr/~besse/Wikistat/Data/ocde.txt",sep='\s+',header=0)
ocde.head()

### 5.3  Statistiques élémentaires
Consulter rapidement ces résultats; Que dire à propos de la symétrie des distributions, de leur normalité, des valeurs atypiques.

In [ ]:
ocde.mean()

In [ ]:
ocde["CNRJ"].hist(bins=20)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(
    ocde, 
    alpha = 0.2, # Transparence des points.
    figsize = (15, 15), # Taille du graphique.
    diagonal = 'kde' # Function de densité.
)

plt.show()

**Q** Quel est le graphique ci-dessous? Que représentent les blocs dagonaux? Que dire des structures de corrélation? 

### 5.4 [Analyse en composantes principales](http://wikistat.fr/pdf/st-m-explo-acp.pdf)
Chaque pays étant observé 4 fois, la principale difficulté technique est de faire apparaître cette structure chronologique dans les graphique afin d'illustrer la dynamique économique de la période considérée.

**Q** Justifier la nécessité de réduire.

**Q** Pourqoi toutes les variables sont des taux?

#### Choix de dimension

In [ ]:
# réduction
ocdeS = preprocessing.scale(ocde)
pca = decomposition.PCA()
cpOcde = pca.fit_transform(ocdeS)
# Eboulis
plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
plt.boxplot(cpOcde)
plt.show()

**Q** Quel est le graphe ci-dessus. Que dire de la première composante? Quelle dimension choisir?

#### Représentation des variables

In [ ]:
coord1 = pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[1]*np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(1, 1, 1)

for i, j, nom in zip(coord1, coord2, ocde.columns):
    plt.text(i, j, nom)
    plt.arrow(0, 0, i, j, color = 'black')

plt.axis((-1.2,1.2,-1.2,1.2))

cercle = plt.Circle((0,0), radius = 1, color = 'gray', fill = False) # Cercle
ax.add_patch(cercle)
plt.show()

**Q** Interpréter chacun des deux premiers axes.

**Exo** représenter le plan (2,3) et interpréter le 3ème axe.

#### Représentation basique des individus

In [ ]:
plt.figure(figsize=(10,6))
for i, j, nom in zip(cpOcde[:,0], cpOcde[:,1], ocde.index):
    plt.text(i, j, nom ,color = "blue")

plt.axis((-5,7,-4,4))  
plt.show()

#### Représentation adaptée à ces données
La structure particulière des données nécessite un graphique adapté. Ceci est en fait le **principal objectif** d'une *bonne exploration des données*: trouver la **représentation graphique** qui permet d'en comprendre toute la structure en une seule vue.

In [ ]:


comp_0 = 0
comp_1 = 1

cmap = plt.get_cmap("tab20")

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(1,1,1)
for i,k in enumerate(np.arange(0,cpOcde.shape[0],4)):

    country = ocde.index[k]
    xs = cpOcde[k:k+4, comp_0]
    ys = cpOcde[k:k+4, comp_1]
    
    ax.plot(xs, ys, color = cmap(i), marker=".", markersize=15)
    
    txt = ax.text(
        xs[-4], 
        ys[-4], 
        country, 
        horizontalalignment = "left", 
        verticalalignment = "top",
        color = cmap(i), 
        fontweight = "bold", 
        fontsize = 15
    )


    ax.set_xlabel(f"PC{comp_0}", fontsize=20)
    ax.set_ylabel(f"PC{comp_1}", fontsize=20)

plt.tight_layout()
plt.show()

**Q** Analyser les évolutions des économies des différents pays. Les remplacer dans la période considérée. 